<a href="https://colab.research.google.com/github/Sudheendra-RD/NLP/blob/main/FakeNewsClassifierLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import pandas as pd

In [73]:
news = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/RNN-LSTM/FakeNewsClassifier/train.csv', index_col=False)
news.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [74]:
del news['author']
del news['id']
del news['text']

In [76]:
news.isnull().sum()

label           0
new column    558
dtype: int64

In [77]:
news = news.dropna()

In [38]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [39]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [40]:
lem = WordNetLemmatizer()

In [113]:
def clean_text(text):
  word = re.sub('[^a-zA-Z]', ' ', str(text))
  word = re.sub('[0-9]', ' ', word)
  word = word.lower()
  word = word.split()
  word = [i for i in word if i not in set(stopwords.words('english'))]
  word = [lem.lemmatize(j) for j in word]

  return corpus

In [110]:
corpus = new_ip['title'].apply(clean_text)

In [111]:
corpus

0        [house dem aide even see comey letter jason ch...
1        [flynn hillary clinton big woman campus breitb...
2                                  [truth might get fired]
3          [civilian killed single u airstrike identified]
4        [iranian woman jailed fictional unpublished st...
                               ...                        
20237          [rapper trump poster child white supremacy]
20238    [n f l playoff schedule matchup odds new york ...
20239    [macy said receive takeover approach hudson ba...
20240          [nato russia hold parallel exercise balkan]
20241                                       [keep f alive]
Name: title, Length: 20242, dtype: object

In [114]:
new_corp = [a for a in corpus]

In [117]:
new_corp[:5]

[['house dem aide even see comey letter jason chaffetz tweeted'],
 ['flynn hillary clinton big woman campus breitbart'],
 ['truth might get fired'],
 ['civilian killed single u airstrike identified'],
 ['iranian woman jailed fictional unpublished story woman stoned death adultery']]

In [44]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [45]:
voc = 5000

In [131]:
a = []
for i in new_corp:
  for j in i:
    a.append(j)

In [133]:
len(a)

20242

In [138]:
word_emb = [one_hot(q,voc) for q in a]

In [139]:
sent_len = 20
embedded = pad_sequences(word_emb, padding='pre', maxlen=sent_len)

In [140]:
embedded[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  509,
       3119, 1124, 4156, 3431,  658,  201, 4894, 4830, 3780], dtype=int32)

In [141]:
# Model Creation
dim = 40
model = Sequential()
model.add(Embedding(voc, dim, input_length=sent_len))
model.add(LSTM(50))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 50)                18200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 218,251
Trainable params: 218,251
Non-trainable params: 0
_________________________________________________________________


In [142]:
len(embedded)

20242

In [147]:
import numpy as np
x = np.array(embedded)
y = np.array(news['label'])

In [148]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size = 0.2, random_state=100
)

In [149]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
254/254 [==============================] - 13s 37ms/step - loss: 0.3167 - accuracy: 0.8760 - val_loss: 0.1766 - val_accuracy: 0.9311
Epoch 2/10
254/254 [==============================] - 8s 33ms/step - loss: 0.1395 - accuracy: 0.9440 - val_loss: 0.1738 - val_accuracy: 0.9318
Epoch 3/10
254/254 [==============================] - 8s 33ms/step - loss: 0.1009 - accuracy: 0.9636 - val_loss: 0.2043 - val_accuracy: 0.9286
Epoch 4/10
254/254 [==============================] - 9s 34ms/step - loss: 0.0743 - accuracy: 0.9745 - val_loss: 0.2254 - val_accuracy: 0.9183
Epoch 5/10
254/254 [==============================] - 9s 35ms/step - loss: 0.0541 - accuracy: 0.9821 - val_loss: 0.2068 - val_accuracy: 0.9133
Epoch 6/10
254/254 [==============================] - 8s 33ms/step - loss: 0.0384 - accuracy: 0.9887 - val_loss: 0.2882 - val_accuracy: 0.9064
Epoch 7/10
254/254 [==============================] - 9s 34ms/step - loss: 0.0260 - accuracy: 0.9926 - val_loss: 0.2974 - val_accuracy: 0.920

In [156]:
predicted = model.predict(x_test)
predicted = (predicted > 0.5)

In [157]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test, predicted)

array([[1886,  198],
       [ 134, 1831]])

In [158]:
accuracy_score(y_test, predicted)

0.9180044455421091